## Imports and Set Up

In [98]:
import os
import random
import warnings
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
from scipy.optimize import minimize
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from colorama import Fore, Style
from IPython.display import clear_output
from lightgbm import LGBMRegressor
from matplotlib import pyplot as plt
from sklearn.base import clone
from sklearn.ensemble import VotingRegressor
from sklearn.impute import KNNImputer
from sklearn.metrics import (accuracy_score, cohen_kappa_score,
                             confusion_matrix, f1_score, mean_absolute_error,
                             mean_squared_error, precision_score, recall_score,
                             classification_report)
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [99]:
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [100]:
def set_seed(seed_value=2024):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed(2024)

## Data Processing

### Load in Files

In [101]:
file_path = '/Users/naliniramanathan/projects/ml_course/final_project/kaggle/input/cmi-piu'

In [102]:
TRAIN_CSV = f'{file_path}/train.csv'
TEST_CSV = f'{file_path}/test.csv'
SAMPLE_SUBMISSION_CSV = f'{file_path}/sample_submission.csv'
SERIES_TRAIN_DIR = f'{file_path}/series_train.parquet'
SERIES_TEST_DIR = f'{file_path}/series_test.parquet'

train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
sample_submission_df = pd.read_csv(SAMPLE_SUBMISSION_CSV)

# Drop all the PCIAT variables as they are not present in the test data
for col in train_df.columns:
    if 'PCIAT' in col:
        train_df.drop(col, axis=1, inplace=True)

In [103]:
# Function to process individual time series files

# Min length of the time series is 927 (hardcoded to speed this up)
def process_time_series(file_name, directory, max_len=927):
    df = pd.read_parquet(os.path.join(directory, file_name, 'part-0.parquet'))
    df['id'] = file_name.split('=')[1]
    # Adjusted this so we get the full parquet - just concatenate all of them
    return df[:max_len]

In [104]:
# Function to load and aggregate time series data
def load_time_series_data(directory):
    file_names = os.listdir(directory)

    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_time_series(fname, directory), file_names),
                            total=len(file_names)))

    return pd.concat(results, ignore_index=True) # This takes slightly longer but ok for now

In [105]:
train_series_df = load_time_series_data(SERIES_TRAIN_DIR)
test_series_df = load_time_series_data(SERIES_TEST_DIR)

100%|██████████| 2/2 [00:00<00:00, 34.99it/s]


In [106]:
train_series_df

,step,X,Y,Z,enmo,anglez,non-wear_flag,light,battery_voltage,time_of_day,weekday,quarter,relative_date_PCIAT,id
0,0,-0.468869,0.412020,-0.236458,0.042506,-19.824650,0.0,27.666666,4179.000000,57480000000000,4,1,28.0,0d01bbf2
1,1,-0.662526,0.533484,0.064034,0.052847,4.300246,0.0,12.666667,4178.666504,57485000000000,4,1,28.0,0d01bbf2
2,2,-0.611384,0.227252,-0.150882,0.060734,-16.545208,0.0,47.000000,4178.333496,57490000000000,4,1,28.0,0d01bbf2
3,3,-0.385799,0.552782,-0.500523,0.070440,-36.452175,0.0,63.799999,4178.000000,57495000000000,4,1,28.0,0d01bbf2
4,4,0.016133,0.031981,-0.825109,0.081058,-67.488388,0.0,6.000000,4177.666504,57500000000000,4,1,28.0,0d01bbf2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923287,922,0.966741,0.028754,0.057921,0.008051,3.353292,0.0,51.564102,4193.000000,46610000000000,5,4,1.0,57de6095
923288,923,1.009239,-0.092948,-0.083056,0.043492,-4.464321,0.0,57.447117,4193.000000,46615000000000,5,4,1.0,57de6095
923289,924,0.986129,-0.272536,-0.248816,0.076904,-13.811750,0.0,63.330128,4193.000000,46620000000000,5,4,1.0,57de6095
923290,925,0.976071,-0.258840,-0.123272,0.069854,-7.051679,0.0,69.213142,4193.000000,46625000000000,5,4,1.0,57de6095


### Encoding of Time Series Data

LTSM Autoencoder taken from: https://machinelearningmastery.com/lstm-autoencoders/

In [107]:
# Encoder Class
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout, seq_len, num_layers):
        super(Encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.seq_len = seq_len
        self.num_layers = num_layers

        self.lstm_enc = nn.LSTM(input_size=input_size, num_layers=num_layers, hidden_size=hidden_size, dropout=dropout, batch_first=True)

    def forward(self, x):
        out, (last_h_state, last_c_state) = self.lstm_enc(x)
        # Get the last hidden state
        x_enc = last_h_state[-1, :, :]         
        return x_enc, out


# Decoder Class
class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout, seq_len, use_act, num_layers):
        super(Decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.seq_len = seq_len
        self.use_act = use_act  # Parameter to control the last sigmoid activation - depends on the normalization used.
        self.act = nn.Sigmoid()

        self.lstm_dec = nn.LSTM(input_size=hidden_size, num_layers=num_layers, hidden_size=hidden_size, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, z):
        z = z.unsqueeze(1).repeat(1, self.seq_len, 1)
        dec_out, (hidden_state, cell_state) = self.lstm_dec(z)
        dec_out = self.fc(dec_out)
        if self.use_act:
            dec_out = self.act(dec_out)

        return dec_out, hidden_state


# LSTM Auto-Encoder Class
class TimeSeriesAutoencoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_ratio, seq_len, num_layers, use_act=True):
        super(TimeSeriesAutoencoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout_ratio = dropout_ratio
        self.seq_len = seq_len

        self.encoder = Encoder(input_size=input_size, num_layers=num_layers, hidden_size=hidden_size, dropout=dropout_ratio, seq_len=seq_len)
        self.decoder = Decoder(input_size=input_size, num_layers=num_layers, hidden_size=hidden_size, dropout=dropout_ratio, seq_len=seq_len, use_act=use_act)

    def forward(self, x, return_last_h=False, return_enc_out=False):
        x_enc, enc_out = self.encoder(x)
        x_dec, last_h = self.decoder(x_enc)

        if return_last_h:
            return x_dec, last_h
        elif return_enc_out:
            return x_dec, enc_out
        return x_dec

In [108]:

# We'll pick the min value as the sequence length
sequence_length = train_series_df['id'].value_counts().min()
train_series_df['id'].value_counts().describe()


count    996.0
mean     927.0
std        0.0
min      927.0
25%      927.0
50%      927.0
75%      927.0
max      927.0
Name: count, dtype: float64

In [109]:
# Sequence length is hardcoded at 927
def prepare_data(df):
    # Convert the dataframe to a numpy array
    ids = df['id'].unique()
    num_samples = len(ids)
    num_features = len(df.columns) - 2

    data = np.array(df.drop(['id', 'step'], axis = 1).values)
    # Convert the numpy array to a PyTorch tensor
    reshaped_data = data.reshape(num_samples, 927, num_features)

    data_tensor = torch.tensor(reshaped_data, dtype=torch.float32)


    return data_tensor, ids

In [110]:
tensor, ids = prepare_data(train_series_df)

In [111]:
tensor.shape

torch.Size([996, 927, 12])

In [112]:
def get_encoded_features(df, lr=0.01, hidden_size=12, num_layers=2, device = 'mps', dropout_ratio=0.2, epochs=100, batch_size=32, sequence_length = 927):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df.drop('id', axis=1))

    # Reshape to df for easier column handling and id tracking
    incl_columns = list(df.columns)
    incl_columns.remove('id')
    print(incl_columns)
    scaled_data_df = pd.DataFrame(scaled_data, columns=incl_columns)
    scaled_data_df['id'] = df['id']
    
    tensor_data, ids = prepare_data(scaled_data_df)
    input_size = tensor_data.shape[2]

    autoencoder = TimeSeriesAutoencoder(input_size, hidden_size, dropout_ratio, sequence_length, num_layers=num_layers)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters(), lr=lr)

    device = torch.device('mps')
    autoencoder = autoencoder.to(device)

    dataset = TensorDataset(tensor_data)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    # Step 5: Training Loop
    autoencoder.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for batch in data_loader:
            batch_data = batch[0].to(device)
            optimizer.zero_grad()
            outputs = autoencoder(batch_data)
            loss = criterion(outputs, batch_data)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_epoch_loss = epoch_loss / len(data_loader)
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Avg Loss: {avg_epoch_loss:.4f}')

    with torch.no_grad():
        tensor_data = tensor_data.to(device)
        encoded_data, vals = autoencoder.encoder(tensor_data)  # Get encoded features

    encoded_data = encoded_data.cpu().numpy()

    encoded_df = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    encoded_df['id'] = ids
    return encoded_df

In [118]:
train_encoded = get_encoded_features(train_series_df, hidden_size=16, num_layers=1, dropout_ratio=0.1)
# test_encoded = get_encoded_features(test_series_df) # Eventually use the same model to encode the test data


['step', 'X', 'Y', 'Z', 'enmo', 'anglez', 'non-wear_flag', 'light', 'battery_voltage', 'time_of_day', 'weekday', 'quarter', 'relative_date_PCIAT']
Epoch [10/100], Avg Loss: 0.8430
Epoch [20/100], Avg Loss: 0.8066
Epoch [30/100], Avg Loss: 0.8017
Epoch [40/100], Avg Loss: 0.7966
Epoch [50/100], Avg Loss: 0.7732
Epoch [60/100], Avg Loss: 0.7590
Epoch [70/100], Avg Loss: 0.7601
Epoch [80/100], Avg Loss: 0.7491
Epoch [90/100], Avg Loss: 0.7464
Epoch [100/100], Avg Loss: 0.7579


In [119]:
train_df = train_df.merge(train_encoded, on='id', how='left')

### Imputation of Missing Numerical Values

In [120]:
# Imputing missing values using KNN imputer
def impute_missing_values(df):
    imputer = KNNImputer(n_neighbors=5)
    numeric_columns = df.select_dtypes(include=['float64','float32','int64']).columns # We treat the time series data as a median for now but others could be used
    imputed_array = imputer.fit_transform(df[numeric_columns])
    imputed_df = pd.DataFrame(imputed_array, columns=numeric_columns)
    for col in df.columns:
        if col not in numeric_columns:
            imputed_df[col] = df[col]
    return imputed_df

In [121]:
train_df.replace([np.inf, -np.inf], np.nan, inplace=True) # Debug why we have inf values

train_df = impute_missing_values(train_df)
train_df['sii'] = train_df['sii'].round().astype(int)

# Imputation is needed in test set for some cases but not others to revisit

print(train_df.isna().sum())

Basic_Demos-Age           0
Basic_Demos-Sex           0
CGAS-CGAS_Score           0
Physical-BMI              0
Physical-Height           0
                       ... 
BIA-Season             1815
PAQ_A-Season           3485
PAQ_C-Season           2239
SDS-Season             1342
PreInt_EduHx-Season     420
Length: 76, dtype: int64


### Categorical Processing

In [122]:
categorical_cols = list(train_df.select_dtypes(include=['object']).columns) #sii (outcome var) is categorical but we are encoding that differently
categorical_cols.remove('id')
print(categorical_cols)

def preprocess_categorical(df):
    for col in categorical_cols:
        df[col] = df[col].fillna('Missing').astype('category')
    return df

train_df = preprocess_categorical(train_df)
test_df = preprocess_categorical(test_df)


train_df = pd.get_dummies(train_df, columns = categorical_cols, drop_first=True, dtype='int')
test_df = pd.get_dummies(test_df, columns = categorical_cols, drop_first=True, dtype='int')

['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'SDS-Season', 'PreInt_EduHx-Season']


In [123]:
# # Feature engineering
# def add_engineered_features(df):
#     df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
#     df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
#     df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
#     df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
#     df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
#     df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
#     df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
#     df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
#     df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
#     df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
#     df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
#     df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
#     df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
#     df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
#     df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
#     return df

In [124]:
# train_df = add_engineered_features(train_df)
# test_df = add_engineered_features(test_df)

In [125]:
train_df = train_df.drop('id', axis=1)
test_df = test_df.drop('id', axis=1)

In [126]:
# encoded_feature_cols = [col for col in train_encoded.columns if col != 'id']
# selected_features = [
#     'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score', 'Physical-BMI',
#     'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
#     'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
#     'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
#     'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone',
#     'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone',
#     'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone',
#     'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
#     'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
#     'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
#     'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
#     'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
#     'BIA-BIA_TBW', 'PAQ_A-PAQ_A_Total', 'PAQ_C-PAQ_C_Total',
#     'SDS-SDS_Total_Raw', 'SDS-SDS_Total_T',
#     'PreInt_EduHx-computerinternet_hoursday', 'BMI_Age', 'Internet_Hours_Age',
#     'BMI_Internet_Hours', 'BFP_BMI', 'FFMI_BFP', 'FMI_BFP', 'LST_TBW',
#     'BFP_BMR', 'BFP_DEE', 'BMR_Weight', 'DEE_Weight', 'SMM_Height',
#     'Muscle_to_Fat', 'Hydration_Status', 'ICW_TBW'
# ] + encoded_feature_cols

In [127]:
# train_df = train_df[selected_features + ['sii']].dropna(subset=['sii'])
# test_df = test_df[selected_features]

## Model 1 - LGBM, XGBoost, CatBoost

In [128]:
def quadratic_weighted_kappa(y_actual, y_predicted):
    return cohen_kappa_score(y_actual, y_predicted, weights='quadratic')

def apply_thresholds(predictions, thresholds):
    return np.where(predictions < thresholds[0], 0,
                    np.where(predictions < thresholds[1], 1,
                             np.where(predictions < thresholds[2], 2, 3)))

def optimize_thresholds(y_true, predictions):
    def loss_func(thresh):
        return -quadratic_weighted_kappa(y_true, apply_thresholds(predictions, thresh))
    initial_thresholds = [0.5, 1.5, 2.5]
    result = minimize(loss_func, initial_thresholds, method='Nelder-Mead')
    return result.x

In [129]:
def train_and_evaluate(model):
    X = train_df.drop('sii', axis=1)
    y = train_df['sii'].astype(int)
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    oof_predictions = np.zeros(len(y))
    # test_predictions = np.zeros(len(test_features))
    train_kappas = []
    val_kappas = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        print(f"Training fold {fold + 1}")
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        print(np.any(np.isnan(X_train)), np.any(np.isnan(y_train)))  # Should be False
        print(np.any(np.isinf(X_train)), np.any(np.isinf(y_train)))  # Should be False

        print(np.any(np.isnan(X_val)), np.any(np.isnan(y_val)))  # Should be False
        print(np.any(np.isinf(X_val)), np.any(np.isinf(y_val)))  # Should be False

        cloned_model = clone(model)
        cloned_model.fit(X_train, y_train)
        
        y_train_pred = cloned_model.predict(X_train).round().astype(int)
        y_val_pred = cloned_model.predict(X_val)

        oof_predictions[val_idx] = y_val_pred

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred)
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred.round().astype(int))

        train_kappas.append(train_kappa)
        val_kappas.append(val_kappa)

        # test_predictions += cloned_model.predict(test_features) / kf.n_splits

        print(f"Fold {fold + 1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Average Train QWK: {np.mean(train_kappas):.4f}")
    print(f"Average Validation QWK: {np.mean(val_kappas):.4f}")

    optimal_thresholds = optimize_thresholds(y, oof_predictions)
    print(f"Optimized Thresholds: {optimal_thresholds}")

    final_oof_predictions = apply_thresholds(oof_predictions, optimal_thresholds)
    # final_test_predictions = apply_thresholds(test_predictions, optimal_thresholds)

    final_kappa = quadratic_weighted_kappa(y, final_oof_predictions)
    print(f"Final Optimized QWK: {Fore.CYAN}{Style.BRIGHT}{final_kappa:.4f}{Style.RESET_ALL}")

In [130]:
lightgbm_params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,
    'lambda_l2': 0.01,
    'n_estimators': 300,
    'random_state': 42,
    'verbose': -1
}

xgboost_params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,
    'reg_lambda': 5,
    'random_state': 42,
    'verbosity': 0
}

# xgboost_params = {
#     'learning_rate': 0.05,
#     'max_depth': 6,
#     'n_estimators': 200,
#     'subsample': 0.8,
#     'colsample_bytree': 0.8,
#     'reg_alpha': 1,
#     'reg_lambda': 5,
#     'random_state': 42,
#     'tree_method': 'gpu_hist',
#     'verbosity': 0
# }

catboost_params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': 42,
    'verbose': 0,
    'l2_leaf_reg': 10
}


# catboost_params = {
#     'learning_rate': 0.05,
#     'depth': 6,
#     'iterations': 200,
#     'random_seed': 42,
#     'verbose': 0,
#     'l2_leaf_reg': 10,
#     'task_type': 'GPU'
# }


In [131]:
lgb_model = LGBMRegressor(**lightgbm_params)
xgb_model = XGBRegressor(**xgboost_params)
cat_model = CatBoostRegressor(**catboost_params)
ensemble_model = VotingRegressor(
    estimators=[('lgb', lgb_model), ('xgb', xgb_model), ('cat', cat_model)],
    weights=[4.0, 4.0, 5.0]
)

In [132]:
train_and_evaluate(ensemble_model)

Average Train QWK: 0.7878
Average Validation QWK: 0.4548
Optimized Thresholds: [0.56900213 1.05590682 2.83650092]
Final Optimized QWK: 0.4971
